We have seen that [using Guassian process can achieve a reasonable visibility approximation](./fov_approx_adv.ipynb) for a single point.
This is achieved by sampling several optical axis directions on the sphere, calculating the visibility at these samples and regressing the visiblity for any rotations. The key is that we need to use the same samples and covariance matrix ($K$) for all the landmarks to regress the visibility of the landmarks in the same way.

In this notebook, we will check how the hyper parameters changse with different landmarks.

In [ ]:
%reset -f
%matplotlib notebook
%reload_ext autoreload
%autoreload 2
import os
import numpy as np
from tqdm import tqdm_notebook

import add_path
import sampler
import visibility as vis
import fov_approx_gp as fgp

import plotly
import plotly.graph_objs as go
from plotly.offline import iplot
plotly.offline.init_notebook_mode(connected=True)

This is to generate the visibility configuration for the Gaussian process approximation, we used [GPy](https://sheffieldml.github.io/GPy/) for the hyperparameters optimization of GP.

In [ ]:
# induced data points for the GP
# 30 50 70 100 120 150
n_induced_fs = 70
# n random landmarks to test
n_train_landmarks = 1000
# FoV
hfov_deg = 45
hfov_rad = np.deg2rad(hfov_deg)
#
sigmoid_k = 15
#
fix_gaussian_noise = 1e-10
#
fast_sigmoid=False

# save basedir
param_dir = '/home/zichao/sources/FIF_ws/src/rpg_information_field/act_map/trace'
assert os.path.exists(param_dir), "change the param_dir to an exiting folder"
abs_save_dir = os.path.join(param_dir, fgp.FoVApproximatorGP.base_savedir_nm,
                            fgp.FoVApproximatorGP.uniqueName(hfov_deg, n_induced_fs, 
                                                             n_train_landmarks, sigmoid_k,
                                                             fast_sigmoid))
if not os.path.exists(abs_save_dir):
    os.makedirs(abs_save_dir)
print("Going to save in {0}".format(abs_save_dir))

# generate necessary data
sample_fs = sampler.sampleSphereUniformArea(n_induced_fs)
rand_lms = sampler.randomPointsOnSphere(n_train_landmarks, 3)
# rand_lms = np.random.uniform(-1, 1, [n_train_landmarks, 3])
# rand_lms = np.array([3 * v / np.linalg.norm(v) for v in rand_lms])

# plotting
print("Let's visualize the sampled camera directions and the random landmarks for training.")
trace_fs = go.Scatter3d(x=sample_fs[:, 0],
                        y=sample_fs[:, 1],
                        z=sample_fs[:, 2],
                        name='Induced Points',
                        mode='markers',
                        marker=dict(color='rgb(0, 80, 200)', size=5))
trace_pw = go.Scatter3d(x=rand_lms[:, 0],
                        y=rand_lms[:, 1],
                        z=rand_lms[:, 2],
                        name='Random Landmarks (training)',
                        mode='markers',
                        marker=dict(color='rgb(150, 150, 150)',
                                    size=3))
layout = go.Layout(showlegend=False)
iplot({"data": [trace_fs, trace_pw]})

In [ ]:
print("Start training: this might take a while, you can reduce the n_train_landmarks if needed.")
hparams = {'rbf_var':[], 'rbf_length':[], 'gauss_var':[]}
for i in tqdm_notebook(range(n_train_landmarks)):
    m = fgp.trainGPFoVApproximatorSingle(sample_fs, hfov_rad,
                                         rand_lms[i],
                                         opt_restart=3, sigmoid_k=sigmoid_k,
                                         fix_gaussian_noise=fix_gaussian_noise,
                                         use_fast_sigmoid=fast_sigmoid)
    hparams['rbf_var'].append(
        np.asscalar(m.rbf.variance.astype(float)))
    hparams['rbf_length'].append(
        np.asscalar(m.rbf.lengthscale))
    hparams['gauss_var'].append(
        np.asscalar(m.Gaussian_noise.variance))

In [ ]:
medians = {}
# plot the hyper parameters
trace_rbf_var = go.Histogram(x=hparams['rbf_var'], name='rbf var')
medians['rbf_var'] = np.median(hparams['rbf_var'])

trace_rbf_length = go.Histogram(x=hparams['rbf_length'], name='rbf len')
medians['rbf_length'] = np.median(hparams['rbf_length'])

trace_gaussian_var = go.Histogram(x=hparams['gauss_var'], name='gaussian n')
medians['gauss_var'] = np.median(hparams['gauss_var'])

print("The medians are:")
display(medians)

fig = plotly.tools.make_subplots(rows=3, cols=1)
fig.append_trace(trace_rbf_var, 1, 1)
fig.append_trace(trace_rbf_length, 2, 1)
fig.append_trace(trace_gaussian_var, 3, 1)
iplot(fig)

In [ ]:
# save the model using the median of the hyperparameters
approx_data = {'sampled_fs': sample_fs, 'sigmoid_k': sigmoid_k, 'hfov_deg': hfov_deg,
              'rbf_var': medians['rbf_var'], 'rbf_lengthscale': medians['rbf_length'],
               'white_var': medians['gauss_var']}
gp_approx = fgp.FoVApproximatorGP()
gp_approx.setApproximatorData(approx_data)
gp_approx.save(abs_save_dir)

print("Saved in {}".format(abs_save_dir))
print("You can use eval_gp_fov_approx.ipynb to validate the approximation")

We can see that the hyper parameters are not very different. Next we will try to [learn one set of parameters from many random landmarks](./gp_fit_all_landmarks.ipynb).